# Create Agents to Research, Write and Edit a Letter of Correspondence

## Setting a Goal

Write a letter of correspondence on an issue from sender to recipient.

In [ ]:
# Install required libraries
!pip install -q 'crewai[tools]' langchain-groq duckduckgo-search

In [ ]:
# Import modules
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
from langchain_groq import ChatGroq
from langchain_community.tools import DuckDuckGoSearchRun
from google.colab import userdata

In [ ]:
# Access Colab secret keys
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [ ]:
# Initialize ChatGroq class with a model
llm = ChatGroq(
    temperature=0,
    groq_api_key=GROQ_API_KEY,
    model_name="llama3-8b-8192"
)

In [ ]:
# Use DuckDuckGo Search component
@tool('DuckDuckGoSearch')
def search(query: str):
    """
    Search the web using DuckDuckGo with an input query.
    """
    return DuckDuckGoSearchRun().run(query)

## Creating Agents

### Agent: Researcher

In [ ]:
researcher = Agent(
    role="Senior Researcher",
    goal="Collect foundational research on {issue}, {sender}, and {recipient}.",
    backstory="An expert researcher who provides valuable insights and recommendations that inform decision making.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

### Agent: Analyst

In [ ]:
analyst = Agent(
    role="Senior Analyst",
    goal="Analyze the research findings about {issue}, {sender}, and {recipient}.",
    backstory="An expert analyst who identifies patterns and trends in the research findings.",
    allow_delegation=False,
    verbose=True,
    llm=llm,
)

### Agent: Writer

In [ ]:
writer = Agent(
    role="Senior Writer",
    goal="Write a professional letter of correspondence on {issue} from {sender} to {recipient}.",
    backstory="You are an expert at writing letters that require careful consideration and attention to detail.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

### Agent: Editor

In [ ]:
editor = Agent(
    role="Senior Editor",
    goal="Edit a professional letter of correspondence on {issue} from {sender} to {recipient}.",
    backstory="An expert editor who ensures the letter of correspondence provides balanced perspectives on opinions and assertions.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

## Creating Tasks

### Task: Research

In [ ]:
research_issue = Task(
    description="Research information on {issue}.",
    expected_output="A bullet point list summarizing the key findings, analysis, and recommendations.",
    agent=researcher,
    async_execution=True,
    tools=[search]
)

In [ ]:
research_sender = Task(
    description="Research {sender}'s statements on {issue}.",
    expected_output="A bullet point list summarizing the key findings, analysis, and recommendations.",
    agent=researcher,
    async_execution=True,
    tools=[search]
)

In [ ]:
research_recipient = Task(
    description="Research {recipient}'s statements on {issue}.",
    expected_output="A bullet point list summarizing the key findings, analysis, and recommendations.",
    agent=researcher,
    async_execution=True,
    tools=[search]
)

### Task: Analyze

In [ ]:
analyze = Task(
    description="Analyze the research findings about {issue}, {sender}, and {recipient}.",
    expected_output="A comprehensive outline for a letter of correspondence on {issue} from {sender} to {recipient}.",
    agent=analyst,
    context=[research_issue, research_sender, research_recipient]
)

### Task: Write

In [ ]:
write = Task(
    description=(
        "1. Write a clear and concise letter of correspondence on {issue} from {sender} to {recipient}.\n"
        "2. Use a formal salutation to greet {recipient} with the proper title and honorifics.\n"
        "3. Use a formal closing, such as 'Sincerely' or 'Yours faithfully' to end the letter.\n"
        "4. Include {sender}'s full name, spaced 3 lines after the closing."
    ),
    expected_output="A professional letter of correspondence in markdown format, including the following sections: formal greeting, purpose, background, proposal, solutions, alternatives, and formal closing.",
    agent=writer,
    context=[analyze]
)

### Task: Edit

In [ ]:
edit = Task(
    description=(
        "1. Edit a letter of correspondence on {issue} from {sender} to {recipient}.\n"
        "2. Ensure the letter is accurate and relevant."
    ),
    expected_output="An edited professional letter of correspondence in markdown format, including the following sections: formal greeting, purpose, background, proposal, solutions, alternatives, and formal closing.",
    agent=editor,
    context=[write],
    output_file="letter.md"
)

## Creating the Crew

In [ ]:
crew = Crew(
    agents=[researcher, analyst, writer, editor],
    tasks=[research_issue, research_sender, research_recipient, analyze, write, edit],
    process=Process.sequential,
    verbose=True
)

## Running the Crew

In [ ]:
issue = "peace"
sender = "Joseph Biden"
recipient = "Vladimir Putin"

In [ ]:
inputs = {
    "issue": issue,
    "sender": sender,
    "recipient": recipient
}

In [ ]:
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Researcher
 [INFO]: == Starting Task: Research information on peace.
 [DEBUG]: == [Senior Researcher] Task output: 


 [DEBUG]: == Working Agent: Senior Researcher
 [INFO]: == Starting Task: Research Joseph Biden's statements on peace.
 [DEBUG]: == [Senior Researcher] Task output: 


 [DEBUG]: == Working Agent: Senior Researcher
 [INFO]: == Starting Task: Research Vladimir Putin's statements on peace.
 [DEBUG]: == [Senior Researcher] Task output: 


 [DEBUG]: == Working Agent: Senior Analyst
 [INFO]: == Starting Task: Analyze the research findings about peace, Joseph Biden, and Vladimir Putin.


> Entering new CrewAgentExecutor chain...


> Entering new CrewAgentExecutor chain...


> Entering new CrewAgentExecutor chain...
Thought: I should start by searching for Vladimir Putin's statements on peace.

Action: DuckDuckGoSearch
Action Input: {"q": "Vladimir Putin statements on peace"} 

I encountered an error while trying to use the tool. This was the e

In [ ]:
from IPython.display import Markdown
Markdown(result)

**[Seal of the President of the United States]**
**[Date]**
**His Excellency Vladimir Putin**
**President of the Russian Federation**
**Moscow, Russian Federation**

Dear President Putin,

I am writing to you today as the President of the United States to discuss the importance of peace and the steps we can take to achieve it. As you know, peace is a fundamental concept that holds immense relevance in our world today. It is essential for the well-being and growth of individuals, communities, and nations.

As you may be aware, I proclaimed Memorial Day as a day of prayer for permanent peace, and I designated the hour of 3:00 PM local time on May 27, 2024, as a national moment of remembrance for peace. This gesture is a testament to the importance of peace and our commitment to promoting it.

I understand that you have expressed openness to peace talks, but with caveats. I want to assure you that we are willing to engage in constructive dialogue to achieve peace. However, we must also acknowledge that peace cannot be achieved through coercion or enforced plans. It requires a genuine willingness to listen, compromise, and work together towards a common goal.

In that spirit, I would like to propose a meeting between our two nations to discuss the possibility of peace talks. I believe that by working together, we can achieve a more peaceful and stable world.

I would like to emphasize the importance of respecting the sovereignty and territorial integrity of nations. We must also recognize the sacrifices made by law enforcement officials and their families, who have given their lives in the line of duty.

Furthermore, I would like to propose the establishment of a joint commission to address the root causes of conflict and promote peaceful resolution mechanisms. This commission would bring together experts from both our nations to identify areas of common interest and work towards finding mutually beneficial solutions.

In conclusion, I believe that peace is a fundamental human right that must be protected and promoted. I am willing to work with you to achieve this goal, and I hope that we can find common ground to move forward.

Thank you for your attention to this matter.

Sincerely,

**Joseph R. Biden Jr.**
**President of the United States of America**

**[Seal of the President of the United States]**

I hope this Final Answer meets your expectations!